In [1]:
import pandas as pd
import numpy as np
from pandas import json_normalize
import requests
import json
import datetime
import tweepy

In [2]:
df = pd.read_csv('llavero.csv')
df

consumer_key  \
0  qTi2G4s8zuybhF5LReyIGvQpx   

                                     consumer_secret  \
0  a0nj2L7DRz3LktHhei4nG7y4ymNnDD7LiIqQmLnw3thzN4...   

                                        bearer_token  \
0  AAAAAAAAAAAAAAAAAAAAAJkHNQEAAAAAqsEpKXKU8XlNgr...   

                                        access_token  \
0  740937600475795456-rDP7E53ht8AalwPMgNMYohUv4VF...   

                             access_token_secret  \
0  yETFBKuwYcKxt6a246GeBOBsNCYBVc46aV5wqzEt3Tx7D   

                          coinmarketcap  
0  71e8ae85-d1b5-45e9-a6e7-fa65d58fabcc

In [3]:

class TweetMiner(object):

    result_limit    =   20    
    data            =   []
    api             =   False
    
    twitter_keys = {
        'consumer_key':        df['consumer_key'][0],
        'consumer_secret':     df['consumer_secret'][0],
        'access_token_key':    df['access_token'][0],
        'access_token_secret': df['access_token_secret'][0]
    }
    
    # Credenciales, creo que no tiene un efecto en el resultado
    def __init__(self, keys_dict=twitter_keys, api=api, result_limit = 20):
        
        self.twitter_keys = keys_dict
        
        auth = tweepy.OAuthHandler(keys_dict['consumer_key'], keys_dict['consumer_secret'])
        auth.set_access_token(keys_dict['access_token_key'], keys_dict['access_token_secret'])
        
        self.api = tweepy.API(auth)
        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user='FernGaminho7', # USER!!!
                         mine_rewteets=False,
                         max_pages=2):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            if last_tweet_id:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                     count=self.result_limit,
                                                     max_id=last_tweet_id - 1,
                                                     tweet_mode = 'extended',
                                                     include_retweets=True
                                                    )        
            else:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                        count=self.result_limit,
                                                        tweet_mode = 'extended',
                                                        include_retweets=True)
                
            for item in statuses:

                mined = {
                    'tweet_id':        item.id,
                    'name':            item.user.name,
                    'screen_name':     item.user.screen_name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.full_text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                    'favourite_count': item.favorite_count,
                    'hashtags':        item.entities['hashtags'],
                    'status_count':    item.user.statuses_count,
                    'location':        item.place,
                    'source_device':   item.source
                }
                
                try:
                    mined['retweet_text'] = item.retweeted_status.full_text
                except:
                    mined['retweet_text'] = 'None'
                try:
                    mined['quote_text'] = item.quoted_status.full_text
                    mined['quote_screen_name'] = status.quoted_status.user.screen_name
                except:
                    mined['quote_text'] = 'None'
                    mined['quote_screen_name'] = 'None'
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data 

In [4]:
miner=TweetMiner(result_limit = 200) #objeto de la clase

In [5]:
mined_tweets = miner.mine_user_tweets(user='FernGaminho7', max_pages=17)

mined_tweets_df= pd.DataFrame(mined_tweets)

In [6]:
mined_tweets_df

tweet_id             name   screen_name  retweet_count  \
0  1366505668925140994  Fernando Gami√±o  FernGaminho7              0   
1  1359945241588092928  Fernando Gami√±o  FernGaminho7              0   
2  1357044556206858243  Fernando Gami√±o  FernGaminho7              0   
3  1352324518832791555  Fernando Gami√±o  FernGaminho7              0   
4  1330760316976771072  Fernando Gami√±o  FernGaminho7            220   
5  1275106926238588928  Fernando Gami√±o  FernGaminho7           5022   
6  1032427850153033728  Fernando Gami√±o  FernGaminho7              3   

                                                text  \
0  @platzi ¬°Tomar descansos! si no lo haces consc...   
1  I'm now a 6 ky≈´ yellow belt on @codewars! Come...   
2  @CarlosLoret ayer cayeron todas las APIs de da...   
3  I just achieved my first rank up on @codewars!...   
4  RT @freddier: El poder m√°s grande para manipul...   
5  RT @deniseramosm: Comparte esta Irma Er√©ndira ...   
6  RT @OVIALCDMX: Al o√≠r una sirena, orille su ve...   

                    mined_at          created_at  favourite_count  \
0 2021-03-08 16:54:18.081201 2021-03-01 21:48:34                1   
1 2021-03-08 16:54:18.081201 2021-02-11 19:19:46                0   
2 2021-03-08 16:54:18.081201 2021-02-03 19:13:29                0   
3 2021-03-08 16:54:18.081201 2021-01-21 18:37:44                0   
4 2021-03-08 16:54:18.081201 2020-11-23 06:29:18                0   
5 2021-03-08 16:54:18.081201 2020-06-22 16:42:36                0   
6 2021-03-08 16:54:18.081201 2018-08-23 00:42:35                0   

                                          hashtags  status_count location  \
0                                               []             7     None   
1  [{'text': 'codewarriors', 'indices': [55, 68]}]             7     None   
2                                               []             7     None   
3  [{'text': 'codewarriors', 'indices': [75, 88]}]             7     None   
4                                               []             7     None   
5                                               []             7     None   
6                                               []             7     None   

        source_device                                       retweet_text  \
0     Twitter Web App                                               None   
1     Twitter Web App                                               None   
2     Twitter Web App                                               None   
3     Twitter Web App                                               None   
4  Twitter for iPhone  El poder m√°s grande para manipular gente es el...   
5  Twitter for iPhone  Comparte esta Irma Er√©ndira de la suerte üçÄ par...   
6  Twitter for iPhone  Al o√≠r una sirena, orille su veh√≠culo y permit...   

  quote_text quote_screen_name  
0       None              None  
1       None              None  
2       None              None  
3       None              None  
4       None              None  
5       None              None  
6       None              None

In [7]:
handle_list= ['APompliano','tyler','michael_saylor','mskvsk','bitso',
              'CryptoWhale','crypto','elonmusk','Kris_HK','CryptoYoda1338']

In [8]:

import time


twitter_dict={}
counter=0

for name in handle_list:
    try:
      twitter_dict[name]=[]
      twitter_dict[name].append(miner.mine_user_tweets(user=name, max_pages=17))
      counter = counter +1
      if counter%40==0:
        time.sleep(900) #15 minute sleep time
    #if name invalid print name and remove key
    except:
      print(name, 'is invalid or locked')
      twitter_dict.pop(name)

In [9]:

mined_tweets_df= pd.DataFrame(twitter_dict)
mined_tweets_df

APompliano  \
0  [{'tweet_id': 1369050920135311362, 'name': 'Po...   

                                               tyler  \
0  [{'tweet_id': 1368972663050416135, 'name': 'Ty...   

                                      michael_saylor  \
0  [{'tweet_id': 1369057194252136448, 'name': 'Mi...   

                                              mskvsk  \
0  [{'tweet_id': 1368547880718467075, 'name': 'Le...   

                                               bitso  \
0  [{'tweet_id': 1369033943736057857, 'name': 'Bi...   

                                         CryptoWhale  \
0  [{'tweet_id': 1368975968552026113, 'name': 'Cr...   

                                              crypto  \
0  [{'tweet_id': 1368959316913500160, 'name': 'Bl...   

                                            elonmusk  \
0  [{'tweet_id': 1369051431903268865, 'name': 'El...   

                                             Kris_HK  \
0  [{'tweet_id': 1367477526814023691, 'name': 'Kr...   

                                      CryptoYoda1338  
0  [{'tweet_id': 1367860132323942401, 'name': 'Cr...

In [10]:
# FUNCION, 
for name in handle_list:
    exec('{} = pd.DataFrame(mined_tweets_df[name][0])'.format(name))

In [12]:
print(handle_list)

['APompliano', 'tyler', 'michael_saylor', 'mskvsk', 'bitso', 'CryptoWhale', 'crypto', 'elonmusk', 'Kris_HK', 'CryptoYoda1338']


In [13]:
frames = [APompliano, tyler, michael_saylor, mskvsk, bitso, CryptoWhale, crypto, elonmusk, Kris_HK, CryptoYoda1338]

In [14]:
result = pd.concat(frames)

In [ ]:
result.shape()

In [ ]:
result['name'].value_counts()

In [17]:
# Exporting as comma-separated variable file
result = result.to_csv('twitter_dataset.csv',index=False)